***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0dd1  GCE       2                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk -U


In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 14 17:04 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '315811133' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('parquet'):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
# doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

In [14]:
s = parquetFile.select('id', 'title').rdd.collectAsMap()

In [20]:
type(s)

dict

In [26]:
import pickle
with open(Path('title')/f'wikiId_titles.pkl', 'wb') as f:
    pickle.dump(s, f)
 
with open(Path('title')/f'wikiId_titles.pkl', 'rb') as f:
    k = pickle.load(f)
k

{4045403: 'Foster Air Force Base',
 4045413: 'Torino Palavela',
 4045419: 'Mad About the Boy',
 4045426: 'Shayne Breuer',
 4045432: 'Parantaka I',
 4045456: 'Arundel (UK Parliament constituency)',
 4045466: 'Andrew Martinez',
 4045471: 'Vancouver VooDoo',
 4045479: 'Invisible plane',
 4045516: 'Shopping channel',
 4045519: 'Turgay (river)',
 4045523: 'Turgay',
 4045525: 'Heinrich Johann Nepomuk von Crantz',
 4045532: 'KUT',
 4045544: 'Dodge Cove',
 4045546: 'Triphone',
 4045554: 'Government House (Nova Scotia)',
 4045575: 'Bud Abell',
 4045577: 'Susette La Flesche',
 4045581: 'Pomme de Terre River (Missouri)',
 4045584: 'Elizabethan Club',
 4045604: 'Ray Abruzzese',
 4045607: 'Holy Family High School (Broomfield, Colorado)',
 4045640: 'Deutsche Gesellschaft für Internationale Zusammenarbeit',
 4045651: 'Ken Adamson',
 4045659: 'Sir William Mackinnon, 1st Baronet',
 4045661: 'List of highest-income ZIP Code Tabulation Areas in the United States',
 4045672: '2nd Battalion, 2nd Marines',


Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
def mysum(ls):
  sum_total =0
  for x in ls:
    sum_total += x
  return sum_total

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "links", 
                    "may", "first", "see",  
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
bucket_name = 'irproject_invertedindex'
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  posting_lists = {}
  w2cnt = Counter(tokens)
  max_tf = 1
  doc_stats = {}
  for w, cnt in w2cnt.items():
    if w not in all_stopwords:
      max_tf = max_tf if max_tf > cnt else cnt
      posting_lists[w] = (id,cnt)
  doc_stats['norm_mtf'] = mysum((x/max_tf)*(x/max_tf) for x in w2cnt.values())**0.5
  doc_stats['norm_dl'] = mysum((x/(len(tokens)+1))*(x/(len(tokens)+1)) for x in w2cnt.values())**0.5
  doc_stats['dl'] = len(tokens)
  doc_stats['max_tf'] = max_tf
  return list(posting_lists.items()), doc_stats

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl)

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda post : (post[0], len(post[1])))

def partition_postings_and_write(base_dir, postings, bucket_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  return postings.map(lambda post: (token2bucket_id(post[0]), post)).groupByKey().map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))
    

In [ ]:
# time the index creation time
t_start = time()
# # word counts map
word_counts_body = parquetFile.select("text", "id").rdd.map(lambda x: (x[1], word_count(x[0], x[1])))
postings_body = word_counts_body.flatMap(lambda x: x[1][0]).groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>20)
w2df_body = calculate_df(postings_filtered_body)
w2df_dict_body = w2df_body.collectAsMap()
doc_stats_body = word_counts_body.map(lambda x: (x[0], x[1][1])).collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write('body_indices', postings_filtered_body, 'inverted_index_body3158').collect()
index_body_const_time = time() - t_start

In [21]:
# time the index creation time
t_start = time()
# # word counts map
word_counts_anchor = parquetFile.select("anchor_text", "id").rdd.map(lambda x: (" ".join([w[1] for w in x[0]]), x[1])).map(lambda x: (x[1], word_count(x[0], x[1])))
postings_anchor = word_counts_anchor.flatMap(lambda x: x[1][0]).groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
w2df_anchor = calculate_df(postings_anchor)
w2df_dict_anchor = w2df_anchor.collectAsMap()
doc_stats_anchor = word_counts_anchor.map(lambda x: (x[0], x[1][1])).collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write('anchor_indices', postings_anchor, 'inverted_index_anchor_3158').collect()
index_const_time_anchor = time() - t_start

In [15]:
# time the index creation time
t_start = time()
# # word counts map
word_counts_title = parquetFile.select("title", "id").rdd.map(lambda x: (x[1], word_count(x[0], x[1])))
postings_title = word_counts_title.flatMap(lambda x: x[1][0]).groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
w2df_title = calculate_df(postings_title)
w2df_dict_title = w2df_title.collectAsMap()
doc_stats_title = word_counts_title.map(lambda x: (x[0], x[1][1])).collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write('inverted_index_body3158', postings_title, 'inverted_index_title_3158').collect()
index_const_time_title = time() - t_start

In [75]:
index_body_const_time

2444.481252193451

In [76]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs('inverted_index_body3158', prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)

Putting it all together

In [77]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_dict_body
# Add the document stats - dictionary for each doc holds max_tf, dl, norm_dl, norm_mtf
inverted_body.DS = doc_stats_body
# write the global stats out
inverted_body.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://inverted_index_body3158/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][347.8 MiB/347.8 MiB]                                                
Operation completed over 1 objects/347.8 MiB.                                    


In [78]:
!gsutil ls -lh $index_dst

 347.8 MiB  2023-01-15T12:38:44Z  gs://inverted_index_body3158/postings_gcp/body_index.pkl
TOTAL: 1 objects, 364691285 bytes (347.8 MiB)


In [45]:
# collect all posting lists locations into one super-set
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs('inverted_index_anchor_3158', prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_anchor[k].extend(v)

In [50]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict_anchor
# Add the document stats - dictionary for each doc holds max_tf, dl, norm_dl, norm_mtf
inverted_anchor.DS = doc_stats_anchor
# write the global stats out
inverted_anchor.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://inverted_index_anchor_3158/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][404.3 MiB/404.3 MiB]                                                
Operation completed over 1 objects/404.3 MiB.                                    


In [51]:
!gsutil ls -lh $index_dst

404.34 MiB  2023-01-14T22:36:09Z  gs://inverted_index_anchor_3158/postings_gcp/anchor_index.pkl
TOTAL: 1 objects, 423979869 bytes (404.34 MiB)


In [43]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs('inverted_index_title_3158', prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_title
# Add the document stats - dictionary for each doc holds max_tf, dl, norm_dl, norm_mtf
inverted_title.DS = doc_stats_title
# write the global stats out
inverted_title.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://inverted_index_title_3158/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][376.4 MiB/376.4 MiB]                                                
Operation completed over 1 objects/376.4 MiB.                                    


In [ ]:
!gsutil ls -lh $index_dst

376.43 MiB  2023-01-14T22:06:04Z  gs://inverted_index_title_3158/postings_gcp/title_index.pkl
TOTAL: 1 objects, 394717109 bytes (376.43 MiB)


In [79]:
len(inverted_body.df)


970349

In [80]:
len(inverted_body.posting_locs)

970349

In [101]:
title_index_src = f'gs://inverted_index_title_3158/postings_gcp/*'
title_index_dst = f'title'
anchor_index_src = f'gs://inverted_index_anchor_3158/postings_gcp/anchor_index.pkl'
anchor_index_dst = f'anchor'
body_index_src = f'gs://inverted_index_body3158/postings_gcp/body_index.pkl'
body_index_dst = f'body'

In [104]:
!mkdir title body anchor 

mkdir: cannot create directory ‘title’: File exists
mkdir: cannot create directory ‘body’: File exists
mkdir: cannot create directory ‘anchor’: File exists


In [102]:
!gsutil -m cp -r  $title_index_src $title_index_dst

Copying gs://inverted_index_title_3158/postings_gcp/0_000.bin...
Copying gs://inverted_index_title_3158/postings_gcp/0_posting_locs.pickle...    
Copying gs://inverted_index_title_3158/postings_gcp/100_000.bin...              
Copying gs://inverted_index_title_3158/postings_gcp/101_000.bin...              
Copying gs://inverted_index_title_3158/postings_gcp/100_posting_locs.pickle...  
Copying gs://inverted_index_title_3158/postings_gcp/101_posting_locs.pickle...  
Copying gs://inverted_index_title_3158/postings_gcp/102_posting_locs.pickle...  
Copying gs://inverted_index_title_3158/postings_gcp/102_000.bin...              
Copying gs://inverted_index_title_3158/postings_gcp/103_000.bin...
Copying gs://inverted_index_title_3158/postings_gcp/103_posting_locs.pickle...  
Copying gs://inverted_index_title_3158/postings_gcp/104_000.bin...              
Copying gs://inverted_index_title_3158/postings_gcp/104_posting_locs.pickle...  
Copying gs://inverted_index_title_3158/postings_gcp/105_00

Copying gs://inverted_index_title_3158/postings_gcp/36_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/36_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/37_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/38_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/37_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/39_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/38_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/40_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/3_000.bin...                
Copying gs://inverted_index_title_3158/postings_gcp/40_000.bin...
Copying gs://inverted_index_title_3158/postings_gcp/41_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/41_000.bin...               
Copying gs://inverted_index_title_3158/post

Copying gs://inverted_index_title_3158/postings_gcp/84_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/85_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/85_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/86_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/86_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/87_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/87_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/88_000.bin...
Copying gs://inverted_index_title_3158/postings_gcp/89_000.bin...               
Copying gs://inverted_index_title_3158/postings_gcp/89_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/88_posting_locs.pickle...   
Copying gs://inverted_index_title_3158/postings_gcp/8_posting_locs.pickle...    
Copying gs://inverted_index_title_3158/post

In [84]:
inv = InvertedIndex.read_index()

NAME
  cp - Copy files and objects


SYNOPSIS

  gsutil cp [OPTION]... src_url dst_url
  gsutil cp [OPTION]... src_url... dst_url
  gsutil cp [OPTION]... -I dst_url



DESCRIPTION
  The ``gsutil cp`` command allows you to copy data between your local file
  system and the cloud, within the cloud, and between
  cloud storage providers. For example, to upload all text files from the
  local directory to a bucket, you can run:

    gsutil cp *.txt gs://my-bucket

  You can also download data from a bucket. The following command downloads
  all text files from the top-level of a bucket to your current directory:

    gsutil cp gs://my-bucket/*.txt .

  You can use the ``-n`` option to prevent overwriting the content of
  existing files. The following example downloads text files from a bucket
  without clobbering the data in your directory:

    gsutil cp -n gs://my-bucket/*.txt .

  Use the ``-r`` option to copy an entire directory tree.
  For example, to upload the directory tree ``dir``

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# # size of index data
# index_dst = f'gs://{bucket_name}/postings_gcp/'
# !gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag